# Avalanche

Source: https://github.com/ContinualAI/avalanche

# Setup

In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git


  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-cuw77wkz
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-cuw77wkz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 532 kB 4.2 MB/s 
     |████████████████████████████████| 121 kB 46.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 25.3 MB/s 
     |████████████████████████████████| 180 kB 55.6 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 139 kB 54.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=414217 sha256=d7925bcec01d981faff793a92d2a8a54f404bf06f3ecb40edebf5e817

# Dataset

In [21]:
from avalanche.benchmarks.classic import SplitMNIST

scenario = SplitMNIST(n_experiences=5)


# Model

In [22]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive, CWRStar, Replay, GDumb, Cumulative, LwF, GEM, AGEM, EWC

model = SimpleMLP(num_classes=scenario.n_classes)
# optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
# criterion = CrossEntropyLoss()
# cl_strategy = Naive(
#     model, optimizer, criterion, 
#     train_mb_size=100, train_epochs=4, eval_mb_size=100
# )

# Training

In [23]:
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.training.plugins import EvaluationPlugin

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=False, stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True))

/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:84: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "
/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:93: UserWarning: No loggers specified, metrics will not be logged
  warnings.warn('No loggers specified, metrics will not be logged')


In [24]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
cl_strategy = Naive(
    model, SGD(model.parameters(), lr=0.001, momentum=0.9),
    CrossEntropyLoss(), train_mb_size=500, train_epochs=1, eval_mb_size=100,
    evaluator=eval_plugin)

In [25]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [5, 6]
Training completed
Computing accuracy on the whole test set
Start of experience:  1
Current Classes:  [1, 2]
Training completed
Computing accuracy on the whole test set
Start of experience:  2
Current Classes:  [0, 8]
Training completed
Computing accuracy on the whole test set
Start of experience:  3
Current Classes:  [9, 3]
Training completed
Computing accuracy on the whole test set
Start of experience:  4
Current Classes:  [4, 7]
Training completed
Computing accuracy on the whole test set


# Evaluation

In [27]:
results

[{'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000': 98.54596806032234,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp001': 99.36087524925021,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp002': 99.01171995880516,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp003': 99.36207859143704,
  'CPUUsage_Exp/eval_phase/test_stream/Task000/Exp004': 100.12730418179868,
  'ConfusionMatrix_Stream/eval_phase/test_stream': tensor([[  0,   0,   0,   0,   0, 628, 352,   0,   0,   0],
          [  0,   0,   0,   0,   0, 716, 419,   0,   0,   0],
          [  0,   0,   0,   0,   0, 236, 796,   0,   0,   0],
          [  0,   0,   0,   0,   0, 932,  78,   0,   0,   0],
          [  0,   0,   0,   0,   0, 298, 684,   0,   0,   0],
          [  0,   0,   0,   0,   0, 803,  89,   0,   0,   0],
          [  0,   0,   0,   0,   0,  12, 946,   0,   0,   0],
          [  0,   0,   0,   0,   0, 950,  78,   0,   0,   0],
          [  0,   0,   0,   0,   0, 798, 176,   0,   0,   0],
          [  0,  